In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
from greenseer.configuration import do_global_configuration
do_global_configuration("config.properties")

- 600519：茅台

In [12]:
from greenseer.dataset.china_dataset import fetch_multi_report, fetch_train_set
from logging.config import fileConfig
import numpy as np
fileConfig('logging_config.ini')
reports = fetch_multi_report(stock_ids=np.array(["600519","600520"]))
reports.shape

(110, 242)

In [14]:
train_set,test_set = fetch_train_set()
train_set.shape,test_set.shape

((456, 242), (87, 242))